In [2]:
import socket
import threading

# --- Encryption / Decryption ---
def encrypt(text, key=3):
    return ''.join(
        chr((ord(c) - 32 + key) % 95 + 32) if c.isprintable() else c
        for c in text
    )

def decrypt(text, key=3):
    return ''.join(
        chr((ord(c) - 32 - key) % 95 + 32) if c.isprintable() else c
        for c in text
    )

# --- Receive Thread ---
def listen_for_messages(sock):
    while True:
        try:
            data, _ = sock.recvfrom(2048)
            print("🔓 " + decrypt(data.decode()))
        except:
            print("❌ Connection lost.")
            break

# --- Setup ---
def start_client():
    server = ("localhost", 65433)
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

    # --- Authentication ---
    user = input("Username: ")
    pwd = input("Password: ")
    auth = f"{user}:{pwd}"
    sock.sendto(auth.encode(), server)

    # --- Wait for Response ---
    try:
        data, _ = sock.recvfrom(2048)
        if data.decode() != "AUTH_SUCCESS":
            print("🚫 Authentication failed.")
            sock.close()
            return
    except:
        print("❌ Could not connect to server.")
        sock.close()
        return

    print("✅ Authenticated! You can now chat. Type 'exit' to quit.\n")

    # --- Start Listener Thread ---
    listener = threading.Thread(target=listen_for_messages, args=(sock,))
    listener.daemon = True
    listener.start()

    # --- Send Loop ---
    try:
        while True:
            msg = input("You: ")
            if msg.lower() == "exit":
                print("👋 Exiting chat.")
                break
            encrypted = encrypt(msg)
            sock.sendto(encrypted.encode(), server)
    finally:
        sock.close()

# --- Run ---
if __name__ == "__main__":
    start_client()


Enter your username:  diueie
Enter your password:  dmedode


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host